# Setup

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# df = pd.concat([pd.read_pickle("./uti/utiTrain.pkl"), pd.read_pickle("./utiTest.pkl")])
# use just training (screening) data
df = pd.read_pickle("./search/searchTrain.pkl")
# Shuffle df
df = df.sample(frac=1).reset_index(drop=True)
df.head()

,code,abstract,scibert
0,0,An algorithm was suggested for identifying hig...,"[0.021734888, 0.07415514, -0.26247355, -0.0595..."
1,0,Purpose The purpose of this paper is to demons...,"[0.44480264, -0.18242139, -0.1497229, 0.367739..."
2,0,There is a lack of quantitative evidence conce...,"[0.20818692, -0.04351063, -0.43295184, 0.53397..."
3,0,Abstract: Effective searching of electronic fu...,"[0.5506647, -0.1169113, -0.27389875, 0.4181351..."
4,0,BACKGROUND: The diagnosis of pulmonary arteria...,"[0.6112776, -0.42844492, -0.3041387, 0.2449172..."


# Helper Functions

In [3]:
# Function to plot confusion matrix
def plotConfMatrix(actual, pred, labels):
    conf_matrix = confusion_matrix(actual, pred)
    labels = ["Exclude", "Include"]
    df_cm = pd.DataFrame(conf_matrix, columns=labels, index=labels)
    df_cm.index.name = 'Actual'
    df_cm.columns.name = 'Predicted'
    plt.figure(figsize = (4,2))
    sns.set(font_scale=1.2)#for label size
    sns.heatmap(df_cm, fmt='d', cmap="Blues", annot=True, annot_kws={"size": 12})# font size
    plt.show()
    return
 
# Function to calculate probabilities of each remaining article
def calcProb(model, initial, remaining):
    # Get initial training data and labels 
    initial_data = initial['scibert'].tolist()
    initial_labels = initial['code'].tolist()

    # Fit model to initial training data
    model.fit(initial_data, initial_labels)

    # Get remaining data for testing
    remaining_data = remaining['scibert'].tolist()

    # Predict probability [exclusion, inclusion] on remaining articles
    pred = model.predict_proba(remaining_data)

    # Calculate score (positive = exclude, negative = include)
    pred = list(map(lambda x: x[0]-x[1], pred))
    # Add probability to dataframe
    remaining['prob'] = pred
    # Sort by probability
    remaining = remaining.sort_values(by=['prob']).reset_index(drop=True)
    
    return remaining

# Calculate the total number of articles screened so far (train / train + test)
def calcPercentScreened(initial, remaining):
    return (initial.index[-1] + 1) / ((remaining.index[-1] + 1) + (initial.index[-1] + 1))

# Calculate the number of articles needed to screen out of entire dataset to find all included articles
# (train + last_index / train + test)
def calcPercentNeedToScreen(initial, remaining, last_index):
    return ((initial.index[-1] + 1) + (last_index + 1)) / ((initial.index[-1] + 1) + (remaining.index[-1] + 1))

# Print stats (return false when all included articles found)
def printStats(initial, remaining):
    # Find index of last included article
    include_indicies = remaining[remaining.code == 1].index
    # Calculate total percentage of articles that need to be screened
    if (len(include_indicies) == 0):
        print("All included articles found after screening: %.2f" % (calcPercentScreened(initial, remaining) * 100))
        return False
    else:
        print("Screened: %.2f Need to screen: %.2f" % (calcPercentScreened(initial, remaining) * 100, calcPercentNeedToScreen(initial, remaining, include_indicies[-1]) * 100))
        return True

# Iterative Predition using Probabilities

In [6]:
# This is good here but bad for the other datasets
# model = LogisticRegression(C=0.05, class_weight='balanced', max_iter=1000)
model = LogisticRegression()

all_data = df['scibert'].tolist()
all_labels = df['code'].tolist()

# Find first included article index
start = df[df.code == 1].index[0] + 1
print("Initial number screened:", start)

# Split df by minimum size to screen
initial = df.iloc[:start,:].reset_index(drop=True)
remaining = df.iloc[start:,:].reset_index(drop=True)

while(remaining.index[-1] > 0):
    remaining = calcProb(model, initial, remaining)
    if(not printStats(initial, remaining)):
        break
    # Take first remaining and append to initial (to account for extra screened article)
    initial = initial.append(remaining.iloc[0], ignore_index=True)
    remaining.drop(0, inplace=True)

Initial number screened: 62
Screened: 4.39 Need to screen: 79.38
Screened: 4.46 Need to screen: 69.60
Screened: 4.54 Need to screen: 80.94
Screened: 4.61 Need to screen: 78.45
Screened: 4.68 Need to screen: 86.53
Screened: 4.75 Need to screen: 83.98
Screened: 4.82 Need to screen: 87.24
Screened: 4.89 Need to screen: 84.69
Screened: 4.96 Need to screen: 84.27
Screened: 5.03 Need to screen: 80.65
Screened: 5.10 Need to screen: 79.80
Screened: 5.17 Need to screen: 71.37
Screened: 5.24 Need to screen: 68.75
Screened: 5.32 Need to screen: 74.70
Screened: 5.39 Need to screen: 70.16
Screened: 5.46 Need to screen: 76.61
Screened: 5.53 Need to screen: 74.56
Screened: 5.60 Need to screen: 71.86
Screened: 5.67 Need to screen: 57.34
Screened: 5.74 Need to screen: 64.78
Screened: 5.81 Need to screen: 65.84
Screened: 5.88 Need to screen: 67.47
Screened: 5.95 Need to screen: 67.12
Screened: 6.02 Need to screen: 68.46
Screened: 6.09 Need to screen: 65.91
Screened: 6.17 Need to screen: 71.44
Screened: 

Screened: 19.91 Need to screen: 34.51
Screened: 19.99 Need to screen: 34.59
Screened: 20.06 Need to screen: 34.51
Screened: 20.13 Need to screen: 34.44
Screened: 20.20 Need to screen: 34.23
Screened: 20.27 Need to screen: 34.23
Screened: 20.34 Need to screen: 34.02
Screened: 20.41 Need to screen: 34.09
Screened: 20.48 Need to screen: 34.16
Screened: 20.55 Need to screen: 34.16
Screened: 20.62 Need to screen: 34.02
Screened: 20.69 Need to screen: 34.02
Screened: 20.77 Need to screen: 34.02
Screened: 20.84 Need to screen: 34.09
Screened: 20.91 Need to screen: 34.09
Screened: 20.98 Need to screen: 34.09
Screened: 21.05 Need to screen: 34.09
Screened: 21.12 Need to screen: 34.09
Screened: 21.19 Need to screen: 33.88
Screened: 21.26 Need to screen: 33.88
Screened: 21.33 Need to screen: 33.81
Screened: 21.40 Need to screen: 33.66
Screened: 21.47 Need to screen: 33.66
Screened: 21.55 Need to screen: 33.66
Screened: 21.62 Need to screen: 33.66
Screened: 21.69 Need to screen: 33.66
Screened: 21